In [1]:
import json
import os

import numpy as np

In [2]:
def align(s1, s2):
    mismatch_score = -3
    match_score = 2
    
    nw = np.full((len(s1) + 1, len(s2) + 1), 0, dtype=np.int16)
    arrow_dict = {}
    
    for i in range(1, nw.shape[1]):
        nw[0, i] = nw[0, i - 1] + mismatch_score
        arrow_dict['{}-{}'.format(0, i)] = ['{}-{}'.format(0, i - 1)]
        
    for i in range(1, nw.shape[0]):
        nw[i, 0] = nw[i - 1, 0] + mismatch_score
        arrow_dict['{}-{}'.format(i, 0)] = ['{}-{}'.format(i - 1, 0)]
        
    for i in range(1, nw.shape[0]):
        for j in range(1, nw.shape[1]):
            horizontal = nw[i, j - 1] + mismatch_score
            vertical = nw[i - 1, j] + mismatch_score
            diagonal = nw[i - 1, j - 1] + match_score

            result = max(horizontal, vertical, diagonal)

            arrow_dict['{}-{}'.format(i, j)] = []
            if result == horizontal:
                arrow_dict['{}-{}'.format(i, j)].append('{}-{}'.format(i, j - 1))
            if result == vertical:
                arrow_dict['{}-{}'.format(i, j)].append('{}-{}'.format(i - 1, j))
            if result == diagonal:
                arrow_dict['{}-{}'.format(i, j)].append('{}-{}'.format(i - 1, j - 1))

            nw[i, j] = result
            
    path = []
    curr_idx = '{}-{}'.format(nw.shape[0] - 1, nw.shape[1] - 1)
    while curr_idx != '0-0':
        prev_idx = arrow_dict[curr_idx][0]
        path.append(curr_idx)
        curr_idx = prev_idx
    path.append('0-0')
    
    idx1 = len(s1) - 1
    idx2 = len(s2) - 1

    s1_align = ''
    s2_align = ''

    for i in range(len(path) - 1):
        curr = [int(i) for i in path[i].split('-')]
        prev = [int(i) for i in path[i + 1].split('-')]

        if (curr[0] > prev[0]) and (curr[1] > prev[1]):  # diagonal
            s1_align += s1[idx1]
            s2_align += s2[idx2]
            idx1 -= 1
            idx2 -= 1
        elif (curr[0] == prev[0]) and (curr[1] > prev[1]):  # horizontal
            s1_align += '-'
            s2_align += s2[idx2]
            idx2 -= 1
        else:  # vertical
            s1_align += s1[idx1]
            s2_align += '-'
            idx1 -= 1

    s1_align = s1_align[::-1]
    s2_align = s2_align[::-1]
    
    return (s1_align, s2_align)

In [3]:
def pdistance(s1, s2):
    differences = 0
    for i, j in zip(s1, s2):
        if i != j:
            differences += 1
    
    mean_lenght = (len(s1) + len(s2)) / 2
    
    return differences / mean_lenght

In [6]:
genus = [f[:-5] for f in os.listdir('raw_data') if f.endswith('.json')]

In [7]:
genus

['Xanthocryptus',
 'Lycus',
 'Camponotus',
 'Polistes',
 'Vespula',
 'Sericomyia',
 'Pepsis',
 'Syrphus',
 'Polyrhachis',
 'Battus',
 'Photinus',
 'Vespa',
 'Calopteron',
 'Danaus',
 'Volucella',
 'Bombus',
 'Myrmecophilus',
 'Messor',
 'Tetramorium',
 'Papilio',
 'Brachypalpus',
 'Syntomeida',
 'Photuris',
 'Merodon',
 'Criorhina',
 'Elytroleptus',
 'Mallota',
 'Argynnis',
 'Xylocopa',
 'Dichocrocis',
 'Hyperechia',
 'Synanthedon',
 'Monomorium',
 'Entypus',
 'Ithomia',
 'Limenitis']

In [8]:
matrix = np.zeros((len(genus), len(genus)))

In [9]:
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
matrix.shape

(36, 36)

In [ ]:
for i in range(3):
    for j in range(3):
        genus1 = genus[i]
        genus2 = genus[j]
        
        file1 = 'raw_data'

In [9]:
with open(files[0], 'r') as f:
    data = json.load(f)

In [22]:
seq1 = data['bold_records']['records']['ASQIC500-09']['sequences']['sequence'][0]['nucleotides']

In [23]:
data['bold_records']['records'].keys()

dict_keys(['ASQIC500-09', 'GBAH21154-19', 'GBAH21156-19', 'GBAH21158-19', 'GBAH21202-19', 'GBAH21223-19', 'GBAH21228-19', 'GBAH21230-19', 'GBAH23669-19', 'GMNZT055-14', 'GMNZW134-14', 'HYQTB122-12', 'NZHH023-12', 'NZHYM158-10', 'NZHYM1647-12'])

In [24]:
data['bold_records']['records']['GMNZW134-14']['sequences']['sequence'][0]['nucleotides']

'ATTCTATATTTTATTTTTGGTATATGAGCTGGTATACTAGGAGCCTCCATAAGATTAATTATTCGTATAGAATTAGGATCAACAGGATCATTAATTGGAAATGATCAAATTTATAATAATTTAGTTACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTGGAAATTGGTTAGTACCATTAATACTTGGAGCTCCAGATATAGCATTTCCACGCATAAATAATATAAGATTTTGATTACTACCCCCTTCCTTATTTCTTTTAATTAATAGAAGATTTATTAATCAAGGAGTAGGAACTGGATGAACTGTTTACCCTCCCTTATCTTTAAATGTTAATCATGAAGGAATATCTGTTGACATAGCAATTTTTTCTTTACATATAGCAGGGATATCTTCTATTATAGGATCAATTAATTTTATTTCAACTATTTTAAATATATACCCCATAAACTCTAAATTAGATCAAATCACTTTATTTACTTGATCAATTTTAATTACAACAATTTTACTTTTATTAGCTGTACCAGTTTTAGCTGGAGCTATTACAATATTATTATCTGATCGTAATATTAAT------------------------------------'

In [25]:
seq2 = data['bold_records']['records']['GMNZW134-14']['sequences']['sequence'][0]['nucleotides']

In [26]:
seq1

'TATTTTATATTTTATTTTTGGAATATGAGCAGGAATATTAGGATCATCAATAAGTTTAATTATCCGAATAGAACTTAGTTCAAGAAGAGGGTTCCTTGATAATAATCTCATTTATAACAATATAGTCACAGCTCATGCTTTTGTCATAATTTTCTTTATAGTTATACCTATTATAATTGGAGGATTTGGTAATTGACTAATCCCTTTAATGTTAGGAGCCCCAGATATAGCCTTTCCTCGTATAAATAATATAAGATTCTGATTATTACCTCCTTCACTATTTTTACTTCTAAATAGAAGAATTATCAATCAAGGAGTAGGAACTGGTTGAACAGTATATCCTCCATTATCATTAAATATTAATCATGAAGGTATATCAGTTGATTTAGCAATTTTTTCTTTACATTTAGCTGGTATATCATCAATTATAGGATCAATCAATTTTATTTCAACAATTTTAAATATATACCCATTAAATTTAAAAATTGATCAAATTTCTTTATTTTCTTGAGCTATTTTTATCACTACAATTTTACTTTTATTAGCCGTACCTGTATTAGCAGGAGCAATTACTATACTATTAGCCGATCGAAATATTAATACATCATTCTTTGATCCTTCAGGTGGAGGAGACCCTATTCTCTACCAACATTTATTT---'

In [28]:
pdistance(seq1, seq2)

0.6815728604471858

In [33]:
seq2

'ATTCTATATTTTATTTTTGGTATATGAGCTGGTATACTAGGAGCCTCCATAAGATTAATTATTCGTATAGAATTAGGATCAACAGGATCATTAATTGGAAATGATCAAATTTATAATAATTTAGTTACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTGGAAATTGGTTAGTACCATTAATACTTGGAGCTCCAGATATAGCATTTCCACGCATAAATAATATAAGATTTTGATTACTACCCCCTTCCTTATTTCTTTTAATTAATAGAAGATTTATTAATCAAGGAGTAGGAACTGGATGAACTGTTTACCCTCCCTTATCTTTAAATGTTAATCATGAAGGAATATCTGTTGACATAGCAATTTTTTCTTTACATATAGCAGGGATATCTTCTATTATAGGATCAATTAATTTTATTTCAACTATTTTAAATATATACCCCATAAACTCTAAATTAGATCAAATCACTTTATTTACTTGATCAATTTTAATTACAACAATTTTACTTTTATTAGCTGTACCAGTTTTAGCTGGAGCTATTACAATATTATTATCTGATCGTAATATTAAT------------------------------------'

In [34]:
align(seq1, seq2)[1]

'ATTCTATATTTTATTTTTGGTATATGAGCTGGTATACTAGGAGCCTCCATAAGATTAATTATTCGTATAGAATTAGGATCAACAGGATCATTAATTGGAAATGATCAAATTTATAATAATTTAGTTACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTGGAAATTGGTTAGTACCATTAATACTTGGAGCTCCAGATATAGCATTTCCACGCATAAATAATATAAGATTTTGATTACTACCCCCTTCCTTATTTCTTTTAATTAATAGAAGATTTATTAATCAAGGAGTAGGAACTGGATGAACTGTTTACCCTCCCTTATCTTTAAATGTTAATCATGAAGGAATATCTGTTGACATAGCAATTTTTTCTTTACATATAGCAGGGATATCTTCTATTATAGGATCAATTAATTTTATTTCAACTATTTTAAATATATACCCCATAAACTCTAAATTAGATCAAATCACTTTATTTACTTGATCAATTTTAATTACAACAATTTTACTTTTATTAGCTGTACCAGTTTTAGCTGGAGCTATTACAATATTATTATCTGATCGTAATATTAAT-------------------------------------------------------------'

In [35]:
pdistance(*align(seq1, seq2))

0.7019667170953101